<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part3_HybridRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Hybrid Recommender

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/models/Content_based_LDA_output.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/models/final_cr_sentiment_data.zip?raw=true 

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/models/final_cr_sentiment_model.pickle?raw=true



--2022-04-12 01:21:31--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-12 01:21:31--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-12 01:21:31--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133,

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 12.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630193 sha256=c3ea45314bd8ea4e05e8c5ee732cea79c2b7631d319c52d1d88cf2aeba08268f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise import dump
import os
from surprise import NormalPredictor
from surprise import SVD
from surprise.accuracy import rmse
import gzip
from collections import defaultdict

In [4]:
def load_model(model_filename):
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    return loaded_model

In [5]:
def get_rec_user(uid, input_df):
    input_id = uid
    data1 = [input_id]
    data2 = input_df['asin'].unique().tolist()

    df = pd.DataFrame(data1)
    df.columns =['reviewerID']

    df1 = pd.DataFrame(data2)
    df1.columns =['asin']
    # filter out reviewed products
    reviewed_product = input_df[input_df['reviewerID'] == input_id].asin.unique().tolist()
    df1 = df1[~df1['asin'].isin(reviewed_product)]

    # Now to perform cross join, we will create
    # a key column in both the DataFrames to
    # merge on that key.
    df['key'] = 1
    df1['key'] = 1

    # to obtain the cross join we will merge
    # on the key and drop it.
    result = pd.merge(df, df1, on ='key')

    result['overall']=0.0
    del result['key']
    return result

In [31]:
def collaborative_SVD_recommender(predictions, product_df, LDA_result, top_num=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
      if iid in LDA_result:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:top_num]

    for uid, user_ratings in top_n.items():
      result = [iid for (iid, _) in user_ratings]

    return result, product_df[product_df['asin'].isin(result)]['ori_title'].tolist()

In [12]:
def topic_modeling_recommender(id, df, top_n=10):
    id = id.lower()

    # get the input product topic number
    topic_num = df[df['asin'].str.lower() == id]['topic_num'].item()

    # remove the input product from the recommendation data
    exclude_input_df = df.copy()
    exclude_input_df = exclude_input_df[exclude_input_df['asin'].str.lower() != id]

    # get the top 10 Probability product for the matching topic number
    output_df = exclude_input_df[exclude_input_df['topic_num'] == topic_num].sort_values('probability', ascending=False).head(top_n)

    # get the product indices
    product_indices = output_df.index.tolist()

    # return the top 10 most similar product
    return df['asin'].iloc[product_indices].tolist(), df['ori_title'].iloc[product_indices].tolist()

In [13]:
product_df = pd.read_csv('./Content_based_LDA_output.zip?raw=true', compression='zip')

In [19]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

asin, title = topic_modeling_recommender(input_id, product_df, 50)

In [20]:
print('Topic Modeling Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, len(asin)):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Topic Modeling Recommender Result for B0001YH10C, coldmate mr 128 mini cooler warmer deluxe mini refrigerator: 
1. B013PRRB4W, Power Pair Special-LG Turbo Series Ultra-Capacity Laundry System with Steam*PURE WHITE COLOR*(WM4270HWA_DLEX4270W)
2. B013PSOBNA, Power Pair Special-LG Turbo Series Ultra-Capacity Laundry System with Steam and Matching Storage Pedestals *GRAPHITE STEEL*(WM4270HVA_DLEX4270V_WDP4V X 2)
3. B013PT1PFQ, Power Pair Special-LG Turbo Series Ultra-Capacity Laundry System with Steam*GRAPHITE STEEL*(WM4270HVA_DLEX4270V)
4. B00HX3ZJKS, PAIR SPECIAL- LG Turbo Series Ultra Capacity Laundry System With Steam Technology (WM3470HVA,DLEX3470V,WDP4V x2)
5. B0049OSUD2, Maytag MFI2665XEM Ice2O 25.5 Cu. Ft. Stainless Steel French Door Refrigerator - Energy Star
6. B00MG225MQ, Power Pair Special- LG Turbo Series Ultra Capacity Laundry System with Steam Technology(WM3570HWA_DLEX3570W)*PURE WHITE IN COLOR*
7. B00MG17WBQ, POWER PAIR SPECIAL-LG TURBO SERIES ULTRA CAPACITY LAUNDRY SYSTEM 

In [34]:
# test the final pretrained model
model_filename = "./final_cr_sentiment_model.pickle?raw=true"
loaded_model = load_model(model_filename)

result_list = []
title_df = pd.read_csv('./cleaned_amazon_product.zip?raw=true', compression='zip')
final_merged_df = pd.read_csv('./final_cr_sentiment_data.zip?raw=true', compression='zip')

input_id = 'A1CY6CQC5HPQGL'
result = get_rec_user(input_id, final_merged_df)

reader = Reader()
valid_Dataset = Dataset.load_from_df(result, reader)

testset = valid_Dataset.df.values.tolist()
predictions = loaded_model.test(testset)

top_num=50
cr_asin, cr_title = collaborative_SVD_recommender(predictions, title_df, asin, top_num)
for i in range(0, len(cr_asin)):
  result_list.append((input_id, cr_asin[i], cr_title[i]))

result_df = pd.DataFrame(result_list)
result_df.columns = ['reviewerID', 'asin', 'ori_title']

reviewer_input_id = 'A1CY6CQC5HPQGL'

print('Collaborative Recommender Result for customer {}: '.format(reviewer_input_id))

for i, product in enumerate(result_df.values.tolist()):
  print('{}. {}, {}'.format(i+1, product[1], product[2]))

IndexError: ignored